In [17]:
import pandas as pd

from catboost import Pool,CatBoostClassifier

In [24]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
original = pd.read_csv('./machine failure.csv')

In [25]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
original.drop('UDI', axis=1, inplace=True)

joined_train_orig = pd.concat([train, original], axis = 0).reset_index(drop=True)
y = joined_train_orig['Machine failure']
X = joined_train_orig.drop(columns=["Machine failure"])

In [26]:
feature_names = ['Product ID','Type',
                 'Air temperature [K]','Process temperature [K]',
                 'Rotational speed [rpm]','Torque [Nm]',
                 'Tool wear [min]','TWF','HDF',
                 'PWF','OSF','RNF']

train_pool = Pool(X.to_numpy(), y.to_numpy(), 
                  feature_names=feature_names, 
                  cat_features=['Product ID', 'Type'])

In [27]:
model = CatBoostClassifier()
model.fit(train_pool, eval_set=train_pool, verbose=False)

test_pool = Pool(test.to_numpy(), feature_names=feature_names, cat_features=['Product ID', 'Type'])
pred = model.predict_proba(test_pool)[:, 1]

In [28]:
sub = pd.read_csv('./sample_submission.csv')
sub['Machine failure'] = pred
sub.to_csv('submission_topper2.csv', index=False)